In [1]:
import sys
print(sys.executable)

C:\Users\Birihan Aschalew\AppData\Local\Programs\Python\Python311\python.exe


In [2]:
!pip install plotly

DEPRECATION: Loading egg at c:\users\birihan aschalew\appdata\local\programs\python\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [4]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import datetime, nltk, warnings
import matplotlib.cm as cm
import itertools

In [6]:
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn import preprocessing, model_selection, metrics, feature_selection
from sklearn.model_selection import GridSearchCV, learning_curve
from sklearn.svm import SVC

In [8]:
from sklearn.metrics import confusion_matrix
from sklearn import neighbors, linear_model, svm, tree, ensemble
from wordcloud import WordCloud, STOPWORDS
from sklearn.ensemble import AdaBoostClassifier
from sklearn.decomposition import PCA
from IPython.display import display, HTML

In [9]:
import plotly.graph_objs as go

In [12]:
from plotly.offline import init_notebook_mode,iplot
init_notebook_mode(connected=True)
warnings.filterwarnings("ignore")
plt.rcParams["patch.force_edgecolor"] = True
plt.style.use('fivethirtyeight')
mpl.rc('patch', edgecolor = 'dimgray', linewidth=1)
%matplotlib inline

In [13]:
# Load and read the dataset
df_init = pd.read_csv('data.csv',encoding="ISO-8859-1", dtype={'CustomerID': str,'InvoiceID': str})
print('Dataframe dimensions:', df_init.shape)

Dataframe dimensions: (541909, 8)


In [14]:
df_init.head(5) # display five rows

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom


In [15]:
df_init.dtypes  # data type


InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID      object
Country         object
dtype: object

In [16]:
df_init.isnull().sum() 

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [18]:
df_init.shape[0]


541909

In [25]:
df_init['InvoiceDate'] = pd.to_datetime(df_init['InvoiceDate'])

col_info = pd.DataFrame(df_init.dtypes).T.rename(index={0:'column type'}) 

col_info = pd.concat([col_info, pd.DataFrame(df_init.isnull().sum()).T.rename(index={0: 'null values (nb)'})])

col_info = pd.concat([col_info, pd.DataFrame(df_init.isnull().sum()/df_init.shape[0]*100).T.rename(index={0:'null values (%)'})])

In [26]:
display(col_info)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
column type,object,object,object,int64,datetime64[ns],float64,object,object
null values (nb),0,0,1454,0,0,0,135080,0
null values (%),0.0,0.0,0.268311,0.0,0.0,0.0,24.926694,0.0


In [27]:
display(df_init[:5])

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


In [28]:
df_init.dropna(axis = 0, subset = ['CustomerID'], inplace = True)
print('Dataframe dimensions:', df_init.shape)

Dataframe dimensions: (406829, 8)


In [30]:
col_info = pd.DataFrame(df_init.dtypes).T.rename(index={0:'column type'})

In [32]:
col_info = pd.concat([col_info, pd.DataFrame(df_init.isnull().sum()).T.rename(index={0: 'null values (nb)'})])

In [33]:
col_info = pd.concat([col_info, pd.DataFrame(df_init.isnull().sum()/df_init.shape[0]*100).T.rename(index={0:'null values (%)'})])

In [34]:
display(col_info)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
column type,object,object,object,int64,datetime64[ns],float64,object,object
null values (nb),0,0,0,0,0,0,0,0
null values (%),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
display(df_init[:10])

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047,United Kingdom


In [36]:
df_init.dropna(axis = 0, subset = ['CustomerID'], inplace = True)
print('Dataframe dimensions:', df_init.shape)

Dataframe dimensions: (406829, 8)


In [39]:
col_info = pd.DataFrame(df_init.dtypes).T.rename(index={0:'column type'})
col_info = pd.concat([col_info, pd.DataFrame(df_init.isnull().sum()).T.rename(index={0:'null values (nb)'})])
col_info = pd.concat([col_info, pd.DataFrame(df_init.isnull().sum()/df_init.shape[0]*100).T.rename(index={0:'null values (%)'})])
display(col_info)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
column type,object,object,object,int64,datetime64[ns],float64,object,object
null values (nb),0,0,0,0,0,0,0,0
null values (%),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
df_init.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID             object
Country                object
dtype: object

In [41]:
df_init.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [42]:
df_init['Country'].duplicated().value_counts()

Country
True     406792
False        37
Name: count, dtype: int64

In [43]:
df_init['InvoiceNo'].duplicated().value_counts()

InvoiceNo
True     384639
False     22190
Name: count, dtype: int64

In [44]:
print('Duplicate Entries: {}'.format(df_init.duplicated().sum()))
df_init.drop_duplicates(inplace = True)

Duplicate Entries: 5225


In [45]:
print("Shape of the data set after droping duplicate entries :", format(df_init.shape))

Shape of the data set after droping duplicate entries : (401604, 8)


In [3]:
import pandas as pd

# Assuming you have a DataFrame called df with the required columns
df_init = pd.DataFrame({'CustomerID': [1, 1, 2, 2],
                        'InvoiceNo': [1001, 1002, 1003, 1004],
                        'Country': ['USA', 'USA', 'UK', 'UK']})

In [5]:
temp_cou = df_init[['CustomerID', 'InvoiceNo', 'Country']].groupby(['CustomerID', 'InvoiceNo', 'Country'])

In [6]:
temp_cou


In [7]:
display(temp_cou
)

In [9]:
# Assuming you have already performed a groupby operation and assigned it to the variable temp_cou
temp_cou_df = temp_cou.size().reset_index(name='count')

In [12]:
# Assuming you have a list of countries
countries = ['USA', 'Canada', 'UK', 'Germany']

In [13]:
countries.index

<function list.index(value, start=0, stop=9223372036854775807, /)>

In [14]:
countries

['USA', 'Canada', 'UK', 'Germany']

In [15]:
data = dict(type='choropleth', 
            locations = countries.index,
            locationmode = 'country names', z = countries,
            text = countries.index, colorbar = {'title':'Order no.'},
            colorscale=[[0, 'rgb(224,255,255)'],
            [0.01, 'rgb(166,206,227)'], [0.02, 'rgb(31,120,180)'],
            [0.03, 'rgb(178,223,138)'], [0.05, 'rgb(51,160,44)'],
            [0.10, 'rgb(251,154,153)'], [0.20, 'rgb(255,255,0)'],
            [1, 'rgb(227,26,28)']],    
            reversescale = False)

In [17]:
import plotly.graph_objects as go

In [19]:
!pip install plotly

DEPRECATION: Loading egg at c:\users\birihan aschalew\appdata\local\programs\python\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [21]:
df_init

,CustomerID,InvoiceNo,Country
0,1,1001,USA
1,1,1002,USA
2,2,1003,UK
3,2,1004,UK


In [24]:
len(df_init['CustomerID'].value_counts())


2

In [25]:
df_init

,CustomerID,InvoiceNo,Country
0,1,1001,USA
1,1,1002,USA
2,2,1003,UK
3,2,1004,UK


In [27]:
from sklearn.decomposition import PCA

In [28]:
!pip install scikit-learn

DEPRECATION: Loading egg at c:\users\birihan aschalew\appdata\local\programs\python\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [31]:
import numpy as np

# Assuming you have a matrix named 'matrix' that you want to scale
# Scale the matrix using some scaling technique (e.g., StandardScaler)
scaled_matrix = your_scaling_function(matrix)

# Now you can proceed with the PCA code
pca = PCA()
pca.fit(scaled_matrix)
pca_samples = pca.transform(scaled_matrix)

NameError: name 'your_scaling_function' is not defined

In [30]:
pca = PCA()
pca.fit(scaled_matrix)
pca_samples = pca.transform(scaled_matrix)

NameError: name 'scaled_matrix' is not defined